In [1]:
import tensorflow_datasets as tfds
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info = True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteGIIEJ2/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteGIIEJ2/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteGIIEJ2/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [2]:
import numpy as np
train_data, test_data = dataset['train'], dataset['test']

train_sentences=[]
test_sentences=[]

train_labels=[]
test_labels=[]

for s,l in train_data:
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())

for s,l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())

vocab_size=10000
embedding_dim = 64
max_length = 150


In [3]:
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")

tokenizer.fit_on_texts(train_sentences)

train_sequences= tokenizer.texts_to_sequences(train_sentences)

padded_train_sequences=pad_sequences(train_sequences, maxlen=max_length, truncating='post', padding="post")

test_sequences= tokenizer.texts_to_sequences(test_sentences)

padded_test_sequences=pad_sequences(test_sequences, maxlen=max_length, truncating='post', padding="post")

In [5]:
import pickle
tokenizer_path = "tokenizer.pickle"
with open(tokenizer_path, "wb") as f:
    pickle.dump(tokenizer, f)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Dropout

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
model.fit(padded_train_sequences, train_labels, epochs=10, validation_data=(padded_test_sequences, test_labels))

Epoch 1/10
782/782 [==============================] - 67s 77ms/step - loss: 0.5157 - acc: 0.7377 - val_loss: 0.4433 - val_acc: 0.7870
Epoch 2/10
782/782 [==============================] - 37s 47ms/step - loss: 0.3502 - acc: 0.8514 - val_loss: 0.5000 - val_acc: 0.7985
Epoch 3/10
782/782 [==============================] - 30s 38ms/step - loss: 0.2823 - acc: 0.8868 - val_loss: 0.5421 - val_acc: 0.7906
Epoch 4/10
782/782 [==============================] - 31s 40ms/step - loss: 0.2225 - acc: 0.9156 - val_loss: 0.6039 - val_acc: 0.7898
Epoch 5/10
782/782 [==============================] - 30s 39ms/step - loss: 0.1718 - acc: 0.9382 - val_loss: 0.6451 - val_acc: 0.7677
Epoch 6/10
782/782 [==============================] - 27s 35ms/step - loss: 0.1264 - acc: 0.9571 - val_loss: 0.7368 - val_acc: 0.7850
Epoch 7/10
782/782 [==============================] - 31s 39ms/step - loss: 0.0994 - acc: 0.9662 - val_loss: 0.7738 - val_acc: 0.7624
Epoch 8/10
782/782 [==============================] - 30s 38ms

In [ ]:
model.save("Sentiment_Analysis.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
